## 1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.

In [1]:
import sqlite3
import pandas as pd


sql_conn = sqlite3.connect('../../data/checking-logs.sqlite')
sql_conn

## 2. Используя только **один** запрос для каждой из групп, создай два датафрейма: `test_results` и `control_results` со столбцами `time` и `avg_diff` и только двумя строками.
- `times` должно иметь значения `before` и `after`.
- `avg_diff` содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
- Учитываются только те пользователи, для которых имеются наблюдения и `before`, и `after`.

In [2]:
test_result = pd.io.sql.read_sql('''
SELECT 'before' as 'times', AVG(diff)
FROM 
		(SELECT uid, AVG(cast((JulianDay(test.first_commit_ts) - 
		JulianDay(dl.deadlines, 'unixepoch'))*24 as integer)) AS diff 
		FROM test 
		LEFT JOIN deadlines as dl, 
			(select uid as id, min(first_view_ts) as min_view
			from test
			group by uid) as min_view
		ON test.labname = dl.labs AND test.uid = min_view.id 
		WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06',
			'laba06s') AND
			test.first_commit_ts < min_view.min_view AND
			test.uid IN
				(SELECT uid 
				FROM test 
				LEFT JOIN deadlines as dl, 
					(select uid as id, min(first_view_ts) as min_view
					from test
					group by uid) as min_view
				ON test.labname = dl.labs AND test.uid = min_view.id 
				WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06',
					'laba06s') AND
					test.first_commit_ts > min_view.min_view
				GROUP BY uid)
		GROUP BY uid)
UNION
SELECT 'after' as 'times', AVG(diff )
FROM 
		(SELECT uid, AVG(cast((JulianDay(test.first_commit_ts) - 
		JulianDay(dl.deadlines, 'unixepoch'))*24 as integer)) AS diff 
		FROM test 
		LEFT JOIN deadlines as dl, 
			(select uid as id, min(first_view_ts) as min_view
			from test
			group by uid) as min_view
		ON test.labname = dl.labs AND test.uid = min_view.id 
		WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06',
			'laba06s') AND
			test.first_commit_ts > min_view.min_view AND 
			test.uid IN 
				(SELECT uid 
				FROM test 
				LEFT JOIN deadlines as dl, 
					(select uid as id, min(first_view_ts) as min_view
					from test
					group by uid) as min_view
				ON test.labname = dl.labs AND test.uid = min_view.id 
				WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06',
					'laba06s') AND
					test.first_commit_ts < min_view.min_view
				GROUP BY uid)
		GROUP BY uid)
''', sql_conn)

In [3]:
control_result = pd.io.sql.read_sql('''
SELECT 'before' as 'times', AVG(diff)
FROM 
		(SELECT uid, AVG(cast((JulianDay(control.first_commit_ts) - 
		JulianDay(dl.deadlines, 'unixepoch'))*24 as integer)) AS diff 
		FROM control 
		LEFT JOIN deadlines as dl, 
			(select uid as id, min(first_view_ts) as min_view
			from control
			group by uid) as min_view
		ON control.labname = dl.labs AND control.uid = min_view.id 
		WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06',
			'laba06s') AND
			control.first_commit_ts < min_view.min_view AND
			control.uid IN
				(SELECT uid 
				FROM control 
				LEFT JOIN deadlines as dl, 
					(select uid as id, min(first_view_ts) as min_view
					from control
					group by uid) as min_view
				ON control.labname = dl.labs AND control.uid = min_view.id 
				WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06',
					'laba06s') AND
					control.first_commit_ts > min_view.min_view
				GROUP BY uid)
		GROUP BY uid)
UNION
SELECT 'after' as 'times', AVG(diff )
FROM 
		(SELECT uid, AVG(cast((JulianDay(control.first_commit_ts) - 
		JulianDay(dl.deadlines, 'unixepoch'))*24 as integer)) AS diff 
		FROM control 
		LEFT JOIN deadlines as dl, 
			(select uid as id, min(first_view_ts) as min_view
			from control
			group by uid) as min_view
		ON control.labname = dl.labs AND control.uid = min_view.id 
		WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06',
			'laba06s') AND
			control.first_commit_ts > min_view.min_view AND 
			control.uid IN 
				(SELECT uid 
				FROM control 
				LEFT JOIN deadlines as dl, 
					(select uid as id, min(first_view_ts) as min_view
					from control
					group by uid) as min_view
				ON control.labname = dl.labs AND control.uid = min_view.id 
				WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06',
					'laba06s') AND
					control.first_commit_ts < min_view.min_view
				GROUP BY uid)
		GROUP BY uid)
''', sql_conn)

In [4]:
test_result

,times,AVG(diff)
0,after,-99.523810
1,before,-66.047619


In [5]:
control_result

,times,AVG(diff)
0,after,-99.322222
1,before,-98.033333


## 3. Мы по-прежнему не используем лабораторное задание 'project1'.

## 4. Закрой соединение.

In [6]:
sql_conn.close()

## 5. Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?

Предположение верно, влияет.